# High School Sports Pull

This file is used to take the historical city championship data from the Calgary Senior High School Athletic Association and convert it into a `.json` file that can be used in the Data for Good - Schools Explorer project.  

# Development Notes

As this project is using the scrum methodology, this is not entirely complete. Several cleaning steps are needed to be completed at this stage.  

Including (but not limited to):
    - `level`, `gender`, and `division` are inconsistent throughout. Espescially from first part to second part.  
    - School names not standardized  
    - Inconsistencies in data entry e.g. sometimes school was entered in `division` instead of `winner` if the sport had no divisions that year  
    - Some records are a tie and more than one school is named in the `winner` field
    

# Import Libraries

In [8]:
# Import Libraries

from docx import Document
import numpy as np
import pandas as pd

# Import Word Document in Raw Form

Obtained from Calgary Senior High School Athletic Association on their website.  
Direct link: http://www.calgaryhighschoolsports.ca/files/city_champs_list_at_june_30_2020.docx  
Retrieved Nov 4, 2022

In [2]:
hs_sports = Document('city_champs_list_at_june_30_2020.docx')

# Document Layout

This word document has historical city champion results in multiple tables. The first table has results from school years 1913-14 to 1998-99.  

The remaining tables contain the data for one year each, from school years 1999-00 to 2019-20.

The first table is the largest, and is in a seperate format than the rest of the tables. We will process this table first, process all remaining tables into another DataFrame, and finally concat the two DataFrames into one.

# Contents

1. First Large Table
2. Remaining Tables
3. Concat all Tables
4. Export results

## First Large Table

The following code uses the docx library to pull the data from the first table into Python and convert it to a Pandas Data Frame.  

As this step takes over 10 minutes to run, the result is saved as a `.cscv` file even though it is an intermediate step and the `.csv` is not needed.

In [5]:
# This pulls the first table into a Data Frame. This table is very large, and this step takes over 10 minutes
table = hs_sports.tables[0]

data = []

keys = None
for i, row in enumerate(table.rows):
    text = (cell.text for cell in row.cells)

    if i == 0:
        keys = tuple(text)
        continue
    row_data = dict(zip(keys, text))
    data.append(row_data)


In [9]:
# Convert to Pandas data drame
df = pd.DataFrame(data)

In [11]:
# Save raw data as .csv
df.to_csv('hs_sports_0_raw.csv')

The table is formatted that there are empty rows when a field is repeated ( e.g. year is null except for the first row of the year. The following codes replace blank fields, with the value of the previous row that is not blank. Then extra rows without a 'WINNER' school are then filtered out.

In [12]:
df['YEAR WON'] = df['YEAR WON'].replace(to_replace = '', method = 'ffill')

In [13]:
df['SPORT'] = df['SPORT'].replace(to_replace = '', method = 'ffill')

In [14]:
df['LEVEL'] = df['LEVEL'].replace(to_replace = '', method = 'ffill')

In [15]:
df['GENDER'] = df['GENDER'].replace(to_replace = '', method = 'ffill')

In [16]:
df['DIVISION'] = df['DIVISION'].replace(to_replace = '', method = 'ffill')

In [19]:
df = df[df.WINNER != ""]

## All Remaining Tables

### 1. Prepare tables for Import into Python

There was an extra column for some rows of the tables for the years 2014-15, 2015-16, 2016-17, 2017-18, and 2018-19  

I cannot figure out how to fix this in Python. I fixed this in Word by doing the following:  

Select View Gridlines from Table Layout menu, so areas with extra cells are visible  

For all years above except 2018-19:  
- Highlight extra cells  
- Delete cells with option of shift cells left  
- Select entire last column, resize entire column to same size  

For year 2018-19:  

- Highlight extra column  
- Delete Extra Column  
- Select entire last column, resize entire column to same size  
Save as 'city_champs_partial_col_removed.docx'  

Note: Method different for year 2018-2019 because table was formatted differently  

### 2. Import Tables from Word doc with Standardized Columns

In [188]:
# All of the tables after the first table are formatted the same. Lets create a list of those tables.
hs_sports_v2 = Document('city_champs_partial_col_removed.docx')
yearly_tables = hs_sports_v2.tables[1:] # Set with index 1: as we want all starting with the second table

### 3. Pull tables into one Pandas DataFrame

In [189]:
# Set Starting Variables
data_2 = []
keys = None
# 1999 - 2000 was the year of the first table in this group.
year_start = 1999 
year_end = 2000
# For loop to run code for all tables 
for table in yearly_tables:
    # The year needs to be specified, as it is not in a column already.
    year_end_short = str(year_end)[2:]
    year_won = f"{year_start}-{year_end_short}" # Standardizes with format in first table.
    # Pulls the text from each cell into a dataframe
    for i, row in enumerate(table.rows):
        text = (cell.text for cell in row.cells)
        # The first row of each table is used to specify the column names
        if i == 0:
            keys = tuple(text)
            continue
        # All other rows the text is pulled into DataFrame
        row_data = dict(zip(keys, text))
        # Column for `YEAR WON` is added
        row_data['YEAR WON'] = year_won
        data_2.append(row_data)
    # Increment the year for the iteration of the next table
    year_start += 1
    year_end += 1

In [190]:
# Convert result into a Pandas DataFrame
data_2 = pd.DataFrame(data_2)

,SPORT,DIVISION,SCHOOL,YEAR WON
0,FOOTBALL,- Jr. Boys Div. I,James Fowler,1999-00
1,,- Jr. Boys Div. II,John G. Diefenbaker,1999-00
2,,- Sr. Boys Div. I,William Aberhart,1999-00
3,,- Sr. Boys Div. II,Bowness,1999-00
4,,,,1999-00
...,...,...,...,...
1505,,Intermediate Girls,did not occur due to Covid19 school closures,2019-20
1506,,Senior Girls,did not occur due to Covid19 school closures,2019-20
1507,,Junior Boys,did not occur due to Covid19 school closures,2019-20
1508,,Intermediate Boys,did not occur due to Covid19 school closures,2019-20


### 4. Some basic cleaning

a. Standardize column names  
b. Fill in rows with missing sport (with previous row based on format of table)  
c. Delete rows with no data (missing winner)  

In [195]:
data_2 = data_2.rename(columns = {'SCHOOL' : 'WINNER'})
data_2['SPORT'] = data_2['SPORT'].replace(to_replace = '', method = 'ffill')

In [200]:
data_2 = data_2[data_2.WINNER != ""]

## Concat all Tables

In [204]:
sports_combined = pd.concat([df, data_2], ignore_index = True)

In [209]:
sports_combined = sports_combined[sports_combined.SPORT != ''] # Removes first row which has no sport listed

## Save Output Files

Outputs saved as both `.csv` and `.json`.

In [210]:
sports_combined.to_csv('sports_combined.csv')

In [211]:
sports_combined.to_json('sports_combined.json')